In [1]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

from pathlib import Path
from datetime import date
from pprint import pprint
from io import BytesIO
from zipfile import ZipFile, BadZipFile
from tqdm import tqdm
import yfinance as yf
import os
from datetime import datetime
import pandas_datareader.data as web
import mplfinance as mpf
import seaborn as sns
import json
import requests
import pandas_datareader.data as web
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

In [2]:
sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_companies = pd.read_html(sp_url, header=0)[0]

In [3]:
sp500_companies

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [4]:
# import yaml
# import nasdaqdatalink
# import quandl
# 
# 
# with open('config.yaml', encoding='UTF-8') as f:
#     _cfg = yaml.load(f, Loader=yaml.FullLoader)
# 
# QUANDL_API_KEY = _cfg['NASDAQ_API_KEY_SECRET']

In [5]:
# # data = quandl.get('FRED/GDP', start_date='2020-12-31', end_date='2023-12-31')
# oil = quandl.get('EIA/PET_RWTC_D', api_key=QUANDL_API_KEY).squeeze()
# oil.plot(lw=2, title='WTI Crude Oil Price', figsize=(15, 5))
# sns.despine()
# plt.tight_layout();

In [6]:
ticker = yf.Ticker('MSFT')
data = ticker.history(period='5d', interval='1m', actions=True, auto_adjust=True)
ticker.calendar

{'Dividend Date': datetime.date(2024, 3, 14),
 'Ex-Dividend Date': datetime.date(2024, 2, 14),
 'Earnings Date': [datetime.date(2024, 4, 23), datetime.date(2024, 4, 29)],
 'Earnings High': 3.06,
 'Earnings Low': 2.78,
 'Earnings Average': 2.83,
 'Revenue High': 62059100000,
 'Revenue Low': 60460000000,
 'Revenue Average': 60797500000}

In [7]:
# get option price
options = ticker.option_chain(ticker.options[0])
options.puts # options.calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,MSFT240308P00245000,2024-02-16 14:30:16+00:00,245.0,0.05,0.00,0.02,0.000000,0.000000,3.0,3,1.718751,False,REGULAR,USD
1,MSFT240308P00250000,2024-02-16 19:01:25+00:00,250.0,0.01,0.00,0.02,0.000000,0.000000,1.0,7,1.656252,False,REGULAR,USD
2,MSFT240308P00255000,2024-02-05 16:37:27+00:00,255.0,0.03,0.00,0.02,0.000000,0.000000,NaN,1,1.593752,False,REGULAR,USD
3,MSFT240308P00260000,2024-02-23 14:36:29+00:00,260.0,0.01,0.00,0.02,0.000000,0.000000,1.0,1,1.531252,False,REGULAR,USD
4,MSFT240308P00265000,2024-02-20 15:42:36+00:00,265.0,0.01,0.00,0.02,0.000000,0.000000,100.0,110,1.468753,False,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,MSFT240308P00455000,2024-01-26 20:21:50+00:00,455.0,51.87,43.20,45.75,0.000000,0.000000,2.0,0,0.000010,True,REGULAR,USD
57,MSFT240308P00460000,2024-03-01 20:49:59+00:00,460.0,44.70,56.40,58.40,0.000000,0.000000,2.0,0,0.593754,True,REGULAR,USD
58,MSFT240308P00480000,2024-02-09 17:36:01+00:00,480.0,60.80,76.35,78.75,0.000000,0.000000,3.0,0,0.900392,True,REGULAR,USD
59,MSFT240308P00520000,2024-03-01 20:56:22+00:00,520.0,104.79,116.40,118.25,0.000000,0.000000,2.0,0,1.559573,True,REGULAR,USD


In [8]:
sns.set_style('whitegrid')

data_path = Path('data')
if not data_path.exists():
    data_path.mkdir()

In [9]:
SEC_URL = 'https://www.sec.gov/'
FSN_PATH = 'files/dera/data/financial-statement-and-notes-data-sets/'

In [10]:
filing_periods = [(d.year, d.quarter) for d in pd.date_range('2019', '2023-03-30', freq='Q')]
filing_periods

[(2019, 1),
 (2019, 2),
 (2019, 3),
 (2019, 4),
 (2020, 1),
 (2020, 2),
 (2020, 3),
 (2020, 4),
 (2021, 1),
 (2021, 2),
 (2021, 3),
 (2021, 4),
 (2022, 1),
 (2022, 2),
 (2022, 3),
 (2022, 4)]

In [11]:
for yr, qtr in tqdm(filing_periods):
    # set (and create) directory
    path = data_path / f'{yr}_{qtr}' / 'source'
    if not path.exists():
        path.mkdir(parents=True)

    # define url and get file
    filing = f'{yr}q{qtr}_notes.zip'
    url = SEC_URL + FSN_PATH + filing
    response = requests.get(url).content

    # decompress and save
    try:
        with ZipFile(BytesIO(response)) as zip_file:
            for file in zip_file.namelist():
                local_file = path / file
                if local_file.exists():
                    continue
                with local_file.open('wb') as output:
                    for line in zip_file.open(file).readlines():
                        output.write(line)
    except BadZipFile:
        print(f'\nBad zip file: {yr} {qtr}\n')
        continue

 12%|█▎        | 2/16 [00:00<00:01, 10.17it/s]


Bad zip file: 2019 1


Bad zip file: 2019 2


Bad zip file: 2019 3


 50%|█████     | 8/16 [00:00<00:00, 18.99it/s]


Bad zip file: 2019 4


Bad zip file: 2020 1


Bad zip file: 2020 2


Bad zip file: 2020 3


Bad zip file: 2020 4


 62%|██████▎   | 10/16 [00:00<00:00, 18.54it/s]


Bad zip file: 2021 1


Bad zip file: 2021 2


Bad zip file: 2021 3


100%|██████████| 16/16 [00:00<00:00, 16.88it/s]


Bad zip file: 2021 4


Bad zip file: 2022 1


Bad zip file: 2022 2


Bad zip file: 2022 3


Bad zip file: 2022 4


In [12]:
for f in tqdm(sorted(list(data_path.glob('**/*.tsv')))):
    # set (and create) directory
    parquet_path = f.parent.parent / 'parquet'
    if not parquet_path.exists():
        parquet_path.mkdir(parents=True)

    # write content to .parquet
    file_name = f.stem  + '.parquet'
    if not (parquet_path / file_name).exists():
        try:
            df = pd.read_csv(f, sep='\t', encoding='utf-8', low_memory=False, error_bad_lines=False)
            df.to_parquet(parquet_path / file_name)
        except Exception as e:
            print(e, ' | ', f)
        # optional: uncomment to delete original .tsv
#         else:
            # f.unlink

0it [00:00, ?it/s]


In [13]:
# file = data_path / '2022_3' / 'source' / '2022q3_notes-metadata.json'
file = data_path / '2020_3' / 'source' / '2020q3_notes-metadata.json'
with file.open() as f:
    data = json.load(f)

pprint(data)

FileNotFoundError: [Errno 2] No such file or directory: 'data/2020_3/source/2020q3_notes-metadata.json'

In [ ]:
sub = pd.read_parquet(data_path / '2020_3' / 'parquet' / 'sub.parquet')
sub.columns

In [ ]:
name = 'APPLE INC'
apple = sub[sub.name == name].T.dropna().squeeze()
key_cols = ['name', 'adsh', 'cik', 'name', 'sic', 'countryba', 'stprba', 'cityba', 'zipba', 'bas1', 'form', 'period','filed']
# apple.loc[key_cols]
apple.loc[key_cols]

In [ ]:
aapl_subs = pd.DataFrame()
for sub in data_path.glob('**/sub.parquet'):
    sub = pd.read_parquet(sub)

    aapl_sub = sub[(sub.cik.astype(int) == apple.cik) & (sub.form.isin(['10-Q', '10-K']))]
    aapl_subs = pd.concat([aapl_subs, aapl_sub])

aapl_subs.form.value_counts()

In [ ]:
aapl_nums = pd.DataFrame()
for num in data_path.glob('**/num.parquet'):
    num = pd.read_parquet(num).drop('dimh', axis=1)
    aapl_num = num[num.adsh.isin(aapl_subs.adsh)]
    print(len(aapl_num))
    aapl_nums = pd.concat([aapl_nums, aapl_num])
aapl_nums.ddate = pd.to_datetime(aapl_nums.ddate, format='%Y%m%d')
aapl_nums.to_parquet(data_path / 'aapl_nums.parquet')

In [ ]:
aapl_nums.info()
aapl_nums

In [ ]:
stock_split = 7
split_date = pd.to_datetime('20180604')

# Filter by tag; keep only values measuring 1 quarter
eps = aapl_nums[(aapl_nums.tag == 'EarningsPerShareDiluted')
                & (aapl_nums.qtrs == 1)].drop('tag', axis=1)

# Keep only most recent data point from each filing
eps = eps.groupby('adsh').apply(lambda x: x.nlargest(n=1, columns=['ddate']))

# Adjust earnings prior to stock split downward
eps.loc[eps.ddate < split_date,'value'] = eps.loc[eps.ddate < split_date, 'value'].div(7)
eps = eps[['ddate', 'value']].set_index('ddate').squeeze().sort_index()
eps = eps.rolling(4,min_periods=4).sum().dropna()
eps.plot(lw=2, figsize=(14, 6), title='Diluted Earnings per Share')
plt.xlabel('')
plt.savefig('diluted eps', dps=300);